In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 500)
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler,MinMaxScaler
import scipy


In [ ]:
train_df=pd.read_csv('contest_train.csv')
test_df =pd.read_csv('contest_test.csv')
print(test_df.shape)
print(train_df.shape)

In [ ]:
count_num = []
for c in [c for c in train_df.columns[2:]]:
    typ = train_df[c].dtype
    uniq = len(np.unique(train_df[c]))
    if uniq>=100: count_num.append(c)

print('Number of numerical features:', len(count_num))

In [ ]:
df_class_0 = train_df[train_df["TARGET"] == 0]
df_class_1 = train_df[train_df["TARGET"] == 1]
df_class_2 = train_df[train_df["TARGET"] == 2]

In [ ]:
df_class_1_over = df_class_1.sample(count_class_0, replace=True)
df_class_2_over = df_class_2.sample(count_class_0, replace=True)

train_df = pd.concat([df_class_0, df_class_1_over, df_class_2_over], axis=0)

In [ ]:
for col in train_df.columns[2:]:
    fig, axs = plt.subplots(ncols=2)
    sns.boxplot(x=train_df[col],ax=axs[0])
    sns.boxplot(x=test_df[col],ax=axs[1])

In [ ]:
def get_redundant_pairs(df):
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlations(df, n=5):
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]

print("Top Absolute Correlations")
abscorr=get_top_abs_correlations(train_df.iloc[:,2:], 700)


In [ ]:
correlation_df=abscorr.reset_index()

In [ ]:
correlation_df=correlation_df[correlation_df[0]>=0.65]

In [ ]:
coeff8_9_10_4=train_df.groupby(['FEATURE_9','FEATURE_10','FEATURE_4'])['FEATURE_8'].agg([('FEATURE_8', 'mean')]).to_dict()
coeff8_9_10_4=list(coeff8_9_10_4.values())
dict_coeff8_9_10_4 = {}
for d in coeff8_9:
    dict_coeff8_9_10_4.update(d)

In [ ]:
for col_number in [150,100,90,80,70,60,50,45,40,35,32,31,30,23,22,11,9,7,6,5,4,3,2,1]:
    print("CN",col_number)
    col=test_df.iloc[:,1:].columns
    low_categorical_list=[]
    for c in col:
        if len(test_df.iloc[:,1:][c].unique())<=col_number:
            if not test_df.iloc[:,1:][c].isna().sum()>0:
                low_categorical_list.append(c)
    print(low_categorical_list)
    for remove_col in high_nans_list:
        if remove_col in low_categorical_list:
                low_categorical_list.remove(remove_col)
                print('remove',remove_col)
    print("CN",col_number,"LEN",len(low_categorical_list))
    cat_list=list(test_df[low_categorical_list].apply(tuple, axis=1))
    for column in high_nans_list:
        coeff=test_df.groupby(low_categorical_list)[column].agg(lambda x: scipy.stats.mode(x)[0]).to_dict()
        test_df[column] = test_df[column].fillna(pd.Series(cat_list).map(coeff))
        print('NA',column,test_df[column].isna().sum())


In [ ]:
mms = MinMaxScaler()

train_df[count_num] = mms.fit_transform(train_df[count_num])
test_df[count_num] = mms.fit_transform(test_df[count_num])
alldf_target=all_df['TARGET']
all_df=all_df.drop(['TARGET'],axis=1)
scaler = StandardScaler()
for high_cat in standscale_list:
    all_df[[high_cat]]=scaler.fit_transform(all_df[[high_cat]])
standart_columns=[]

for column in all_df.columns:
    if all_df[column].describe()['std']>1:
        standart_columns.append(column)
for column in standart_columns:
    all_df[column]=scaler.fit_transform(all_df[[column]])
Y_train=train_df['TARGET']
X_train=all_df[:24416]
X_test=all_df[24416:]
print(Y_train.shape,X_train.shape,X_test.shape)
X_train, x_valid, y_train, y_valid = train_test_split(X_train, Y_train, test_size=0.1, random_state=0)
cv_res = pd.DataFrame({"CrossValMeans":cv_means,"CrossValerrors": cv_std,"Algorithm":["GradientBoostingClassifier","SVC","RandomForestClassifier","ExtraTreesClassifier",'KNeighborsClassifier',"LogisticRegression"]})
g = sns.barplot("CrossValMeans","Algorithm",data = cv_res, palette="Set3",orient = "h",**{'xerr':cv_std})
g.set_xlabel("Mean Accuracy")
g = g.set_title("Cross validation scores")
GBC = GradientBoostingClassifier()
kfold = StratifiedKFold(n_splits=10)
gb_param_grid = {'loss' : ["deviance"],
              'n_estimators' : [300],
              'learning_rate': [0.01],
              'max_depth': [8],
              'min_samples_leaf': [150],
              'max_features': [0.1] 
              }
gsGBC = GridSearchCV(GBC,param_grid = gb_param_grid, cv=kfold, scoring="f1_macro", n_jobs= 4, verbose = 1)
gsGBC.fit(X_train,y_train)
GBC_best = gsGBC.best_estimator_
GB_fit = gsGBC.fit(X_train, y_train)
prediction = pd.Series(GB_fit.predict(X_test), name="Predicted")
results = pd.concat([ss['ID'],prediction],axis=1)